# IMU Data Classification

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

from helper_filter import *

## Filter data
### Extract data tables
Read the raw data from each target action and preview data.

In [15]:
def load_data(folder_path):
    file_list = glob.glob(folder_path + "/*.dat") 
    files =[]

    for i in range(1, len(file_list)): 
        df = pd.read_csv(file_list[i], delimiter='\t', header=None) 

        new_headers = [f"{df.iloc[0][j]}_{df.iloc[1][j]}"
                       .replace("Wide Range Accelerometer", "Accelerometer")
                       .replace(" ", "_") 
                       for j in range(0, len(df.columns))]
        df.columns = new_headers
        df = df.drop([0, 1, 2, 3])
        df.reset_index(drop=True, inplace=True)

        files.append(df)

    return files

lqw_raw = load_data("./IMU_Data/LGW")

In [16]:
lqw_raw[0].head()

,Thigh_R_Timestamp,Thigh_R_Gyroscope X,Thigh_R_Gyroscope Y,Thigh_R_Gyroscope Z,Thigh_R_Wide Range Accelerometer X,Thigh_R_Wide Range Accelerometer Y,Thigh_R_Wide Range Accelerometer Z,Thigh_R_Magnetometer X,Thigh_R_Magnetometer Y,Thigh_R_Magnetometer Z,...,Pelvis_Gyroscope Y,Pelvis_Gyroscope Z,Pelvis_Wide Range Accelerometer X,Pelvis_Wide Range Accelerometer Y,Pelvis_Wide Range Accelerometer Z,Pelvis_Magnetometer X,Pelvis_Magnetometer Y,Pelvis_Magnetometer Z,Annotation_Pulse,Annotation_Level
0,241787.1094,0.5345,1.2220,-0.8404,-2.1075,9.1086,3.2014,-0.2819,-0.7146,-0.2299,...,0.5600,0.0212,-0.1209,9.6397,-1.0211,-0.1688,-0.6494,-0.1636,0.0000,0.0000
1,241796.8750,0.5345,1.2220,-0.8404,-2.1090,9.0322,3.0518,-0.2819,-0.7223,-0.2321,...,0.5600,0.0212,-0.1209,9.7162,-1.0218,-0.1725,-0.6264,-0.1728,0.0000,0.0000
2,241806.6406,-0.2664,0.2175,-1.1036,-2.1872,9.1850,3.2015,-0.2800,-0.7049,-0.2148,...,0.5600,0.0212,-0.1209,9.7926,-1.0226,-0.1781,-0.6533,-0.1751,0.0000,0.0000
3,241816.4062,-0.2664,0.2175,-1.1036,-2.1082,9.1850,3.1255,-0.2876,-0.7049,-0.2169,...,1.0746,-0.0255,-0.2044,9.7123,-1.4106,-0.1744,-0.6513,-0.1728,0.0000,0.0000
4,241826.1719,-0.2664,0.2175,-1.1036,-2.0293,9.0322,3.0510,-0.2743,-0.6893,-0.2169,...,1.0746,-0.0255,-0.2793,9.7934,-0.9448,-0.1725,-0.6284,-0.1728,0.0000,0.0000


## Preprocess data
### Apply filtering

In [ ]:
# TODO 

### Apply the slinding window technique

In [ ]:
# TODO

## Train models
### ANN

In [ ]:
# TODO

### SVM

In [ ]:
# TODO

### CNN

In [ ]:
# TODO

### Comparison

In [ ]:
# TODO